In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier





In [15]:
df = pd.read_csv('merged_21.csv')

In [16]:
df = pd.read_csv('merged_21.csv')
del df['id']

mean_popularity = 50
df["popularity"] = [ 1 if i >= mean_popularity else 0 for i in df.popularity ]
df


,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,popularity,speechiness,...,valence_yr,popularity_yr,mode,key_0_yr,key_1_yr,key_2_yr,key_3_yr,key_4_yr,key_5_yr,key_6_yr
0,0.013100,0.2560,182347,0.895,0,0.000106,0.0821,-4.860,0,0.0707,...,0.583424,35.272231,1,0,0,1,0,0,0,0
1,0.980000,0.2770,206972,0.145,0,0.879000,0.1110,-19.898,0,0.0845,...,0.432251,3.672500,1,1,0,0,0,0,0,0
2,0.795000,0.6850,314667,0.483,0,0.878000,0.1130,-10.202,0,0.0337,...,0.447291,7.707000,1,0,0,0,1,0,0,0
3,0.656000,0.7880,179747,0.808,0,0.000000,0.1540,-6.590,0,0.0395,...,0.447291,7.707000,1,0,0,0,1,0,0,0
4,0.302000,0.0753,498560,0.150,0,0.884000,0.1210,-16.705,0,0.0371,...,0.443625,3.419500,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172225,0.000067,0.5670,37000,0.862,0,0.875000,0.4810,-5.617,0,0.0670,...,0.426566,27.599484,1,0,0,0,0,1,0,0
172226,0.000994,0.3960,501824,0.979,0,0.609000,0.1360,-9.138,0,0.0878,...,0.438681,29.107904,1,1,0,0,0,0,0,0
172227,0.864000,0.6380,161560,0.156,0,0.000000,0.1830,-17.372,0,0.0741,...,0.490335,19.231500,1,1,0,0,0,0,0,0
172228,0.029900,0.5330,342827,0.547,0,0.011300,0.0723,-9.290,0,0.0326,...,0.593672,33.923039,1,0,0,1,0,0,0,0


In [17]:
X = df[df.columns.difference(['popularity'])]
y = pd.cut(x=df["popularity"], bins=[-1,42,100], labels=[0,1])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [18]:

model = RandomForestClassifier()

clf = RandomForestClassifier(random_state=31).fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [19]:

n_estimators = [int(x) for x in np.linspace(start = 200, stop = 1000, num = 5)] # Number of Decisions Trees in the forest
max_features = ['auto', 'sqrt']                                                 # Max features used to split at each node
max_depth = np.linspace(20,100,5).tolist()                                      # Max depth of each tree
max_depth.append(None) 
bootstrap = [True, False]
min_samples_split = [2, 5, 10]                                                  # Minimum samples required to split at a node
min_samples_leaf = [1, 2, 4]


# Parameter Grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}



rfc=RandomForestClassifier(random_state=31)
random_rfc = RandomizedSearchCV(estimator = rfc, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, 
                                random_state=42, n_jobs = -1)
random_rfc.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
264 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
57 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1144, in wrapper
    estimator._validate_params()
  File "/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 637, in _validate_params
    validate_para

RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=31),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [20.0, 40.0, 60.0, 80.0,
                                                      100.0, None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000]},
                   random_state=42, verbose=2)

In [21]:
random_rfc.best_params_

{'n_estimators': 400,
 'min_samples_split': 2,
 'min_samples_leaf': 2,
 'max_features': 'sqrt',
 'max_depth': None,
 'bootstrap': False}

In [20]:
rnd_clf=RandomForestClassifier(n_estimators=400, max_depth=20, max_features='sqrt', criterion='gini', min_samples_leaf=4,
                               min_samples_split=2, bootstrap=False, n_jobs=-1)
rnd_clf.fit(X_train, y_train)
y_pred_rf=rnd_clf.predict(X_test)

In [22]:
print('\033[1mClassification Report')
print('\033[0m')
print(classification_report(y_test, y_pred_rf))

Classification Report

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     34446

    accuracy                           1.00     34446
   macro avg       1.00      1.00      1.00     34446
weighted avg       1.00      1.00      1.00     34446



In [23]:
from xgboost import XGBClassifier
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report

In [24]:
df = pd.read_csv('merged_21.csv')
del df['id']
mean_popularity = 50
df["popularity"] = [ 1 if i >= mean_popularity else 0 for i in df.popularity ]
df["popularity"].head()

0    0
1    0
2    0
3    0
4    0
Name: popularity, dtype: int64

In [25]:
X  = df.drop(['popularity'], axis=1)
y = df['popularity']

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=35)


In [27]:
xgb_cl = XGBClassifier(loss='binary_crossentropy', max_iter=1000, random_state=42)


In [28]:
params = {'n_estimators': [50, 100, 200], 
          'learning_rate': [0.01, 0.1, 0.2], 
          'objective': ['binary:logistic'], 
         'max_depth': [3, 5, 7]}

In [29]:
xgb_clf = GridSearchCV(xgb_cl, params, cv=5)

In [30]:
xgb_clf.fit(X_train, y_train)


/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [01:17:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "loss", "max_iter" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [01:17:30] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "loss", "max_iter" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [01:17:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "loss", "max_iter" } are not used.

  warnings.warn(smsg, UserWarning)
/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [01:17:31] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "loss", "max_iter" } are not used.

  warnings.warn

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_cat_threshold=None,
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_iter=1000, max_leaves=None,
                                     min_child_weight=None, missing=nan,
                                     monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, ...),
             param_grid={'learning_rate': [0.01, 0.1, 0.2],
                         'max_depth': [3, 5, 7], 'n_estimators': [50, 100, 200],
                         'objective': ['binary:logistic']})

In [31]:
y_pred_xgb = xgb_clf.predict(X_test)

In [32]:
print('\033[1mClassification Report')
print('\033[0m')
print(classification_report(y_test, y_pred_xgb))

Classification Report

              precision    recall  f1-score   support

           0       0.93      0.98      0.96     28684
           1       0.85      0.66      0.74      5762

    accuracy                           0.92     34446
   macro avg       0.89      0.82      0.85     34446
weighted avg       0.92      0.92      0.92     34446



In [33]:
from sklearn.linear_model import Perceptron


from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Perceptron
from sklearn.metrics import accuracy_score

In [34]:
df = pd.read_csv('merged_21.csv')
del df['mode']
del df['id']

mean_popularity = 50
df["popularity"] = [ 1 if i >= mean_popularity else 0 for i in df.popularity ]
df


,acousticness,danceability,duration_ms,energy,explicit,instrumentalness,liveness,loudness,popularity,speechiness,...,tempo_yr,valence_yr,popularity_yr,key_0_yr,key_1_yr,key_2_yr,key_3_yr,key_4_yr,key_5_yr,key_6_yr
0,0.013100,0.2560,182347,0.895,0,0.000106,0.0821,-4.860,0,0.0707,...,120.209319,0.583424,35.272231,0,0,1,0,0,0,0
1,0.980000,0.2770,206972,0.145,0,0.879000,0.1110,-19.898,0,0.0845,...,110.008113,0.432251,3.672500,1,0,0,0,0,0,0
2,0.795000,0.6850,314667,0.483,0,0.878000,0.1130,-10.202,0,0.0337,...,108.561912,0.447291,7.707000,0,0,0,1,0,0,0
3,0.656000,0.7880,179747,0.808,0,0.000000,0.1540,-6.590,0,0.0395,...,108.561912,0.447291,7.707000,0,0,0,1,0,0,0
4,0.302000,0.0753,498560,0.150,0,0.884000,0.1210,-16.705,0,0.0371,...,109.569882,0.443625,3.419500,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172225,0.000067,0.5670,37000,0.862,0,0.875000,0.4810,-5.617,0,0.0670,...,123.644288,0.426566,27.599484,0,0,0,0,1,0,0
172226,0.000994,0.3960,501824,0.979,0,0.609000,0.1360,-9.138,0,0.0878,...,124.621497,0.438681,29.107904,1,0,0,0,0,0,0
172227,0.864000,0.6380,161560,0.156,0,0.000000,0.1830,-17.372,0,0.0741,...,112.511724,0.490335,19.231500,1,0,0,0,0,0,0
172228,0.029900,0.5330,342827,0.547,0,0.011300,0.0723,-9.290,0,0.0326,...,119.346342,0.593672,33.923039,0,0,1,0,0,0,0


In [35]:
df.columns

Index(['acousticness', 'danceability', 'duration_ms', 'energy', 'explicit',
       'instrumentalness', 'liveness', 'loudness', 'popularity', 'speechiness',
       'tempo', 'valence', 'year', 'key_0', 'key_1', 'key_2', 'key_3', 'key_4',
       'key_5', 'key_6', 'key_7', 'key_8', 'key_9', 'key_10', 'key_11',
       'minor', 'major', 'acousticness_ar', 'danceability_ar',
       'duration_ms_ar', 'energy_ar', 'instrumentalness_ar', 'liveness_ar',
       'loudness_ar', 'speechiness_ar', 'tempo_ar', 'valence_ar',
       'popularity_ar', 'count', 'key_0_ar', 'key_1_ar', 'key_2_ar',
       'key_3_ar', 'key_4_ar', 'key_5_ar', 'key_6_ar', 'key_7_ar', 'key_8_ar',
       'key_9_ar', 'key_10_ar', 'key_11_ar', 'minor_ar', 'major_ar',
       'acousticness_yr', 'danceability_yr', 'duration_ms_yr', 'energy_yr',
       'instrumentalness_yr', 'liveness_yr', 'loudness_yr', 'speechiness_yr',
       'tempo_yr', 'valence_yr', 'popularity_yr', 'key_0_yr', 'key_1_yr',
       'key_2_yr', 'key_3_yr', 'key_4_yr',

In [36]:
y = df["popularity"].values
X = df.drop(["popularity"],axis=1)

In [37]:
input_scaler = StandardScaler()
X_normalized = input_scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.30, random_state=42, stratify=y
)

In [38]:
scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X.shape

(172230, 70)

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42) 


In [40]:

model = Perceptron()

clf = Perceptron(random_state=0).fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [41]:

params = {'alpha': [0.0001, 0.001, 0.01 , 0.1 ]}

clf1 = GridSearchCV(model, {
    'eta0' : [ 0.0001, 0.001, 0.01, 0.1], 'max_iter' : [10, 100, 1000, 10000]}, cv = 2, return_train_score = False)

clf1.fit(X_train , y_train)
clf1.cv_results_


/Users/vijayramaraju/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_stochastic_gradient.py:713: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  warnings.warn(


{'mean_fit_time': array([0.19865465, 0.18026304, 0.159971  , 0.11696804, 0.1235199 ,
        0.13862514, 0.13611853, 0.12722456, 0.1124984 , 0.12370396,
        0.15319049, 0.16830492, 0.16978884, 0.19447792, 0.21553481,
        0.24345052]),
 'std_fit_time': array([0.03349447, 0.00428581, 0.0127449 , 0.00356805, 0.00608516,
        0.01044202, 0.00076759, 0.00137246, 0.01422966, 0.00875115,
        0.01584351, 0.00853205, 0.05135798, 0.07754004, 0.07022703,
        0.05154145]),
 'mean_score_time': array([0.01065743, 0.00695634, 0.0098561 , 0.00811386, 0.00652349,
        0.00828958, 0.00996745, 0.00697196, 0.00971961, 0.01004207,
        0.01175439, 0.00917947, 0.00854456, 0.01195753, 0.01693916,
        0.0155915 ]),
 'std_score_time': array([0.00208437, 0.0003655 , 0.00038397, 0.00335383, 0.00174034,
        0.00172544, 0.00541747, 0.00224197, 0.00211143, 0.00206506,
        0.00055063, 0.00203168, 0.00384748, 0.00726259, 0.00634503,
        0.0097307 ]),
 'param_eta0': masked_arra

In [42]:
# best at 


df1 = pd.DataFrame(clf1.cv_results_)
df1

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_eta0,param_max_iter,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,0.198655,0.033494,0.010657,0.002084,0.0001,10,"{'eta0': 0.0001, 'max_iter': 10}",0.875689,0.873338,0.874514,0.001176,1
1,0.180263,0.004286,0.006956,0.000365,0.0001,100,"{'eta0': 0.0001, 'max_iter': 100}",0.875689,0.873338,0.874514,0.001176,1
2,0.159971,0.012745,0.009856,0.000384,0.0001,1000,"{'eta0': 0.0001, 'max_iter': 1000}",0.875689,0.873338,0.874514,0.001176,1
3,0.116968,0.003568,0.008114,0.003354,0.0001,10000,"{'eta0': 0.0001, 'max_iter': 10000}",0.875689,0.873338,0.874514,0.001176,1
4,0.123520,0.006085,0.006523,0.001740,0.001,10,"{'eta0': 0.001, 'max_iter': 10}",0.875689,0.873338,0.874514,0.001176,1
5,0.138625,0.010442,0.008290,0.001725,0.001,100,"{'eta0': 0.001, 'max_iter': 100}",0.875689,0.873338,0.874514,0.001176,1
6,0.136119,0.000768,0.009967,0.005417,0.001,1000,"{'eta0': 0.001, 'max_iter': 1000}",0.875689,0.873338,0.874514,0.001176,1
7,0.127225,0.001372,0.006972,0.002242,0.001,10000,"{'eta0': 0.001, 'max_iter': 10000}",0.875689,0.873338,0.874514,0.001176,1
8,0.112498,0.014230,0.009720,0.002111,0.01,10,"{'eta0': 0.01, 'max_iter': 10}",0.875689,0.873338,0.874514,0.001176,1
9,0.123704,0.008751,0.010042,0.002065,0.01,100,"{'eta0': 0.01, 'max_iter': 100}",0.875689,0.873338,0.874514,0.001176,1


In [43]:
# the best : {'eta0': 0.01, 'max_iter': 10000}

In [44]:
clf = Perceptron(random_state=0, max_iter=10000, eta0 = 0.01).fit(X_train, y_train)
y_pred = clf.predict(X_train)

In [45]:
accuracy_score(y_train, y_pred)


0.8754717528885793

In [46]:
y_pred = clf.predict(X_test)


In [47]:
accuracy_score(y_pred, y_test)


0.8764152586657377

In [48]:
print(classification_report(y_test, y_pred))
print("Classification report\n")


              precision    recall  f1-score   support

           0       0.92      0.93      0.93     28768
           1       0.63      0.62      0.62      5678

    accuracy                           0.88     34446
   macro avg       0.78      0.77      0.77     34446
weighted avg       0.88      0.88      0.88     34446

Classification report

